In [13]:
import numpy as np
from sympy import Rational, Poly
from sympy.abc import z


In [11]:
la = [Rational(1,2), 1, -Rational(1/2)]
ha = [Rational(1/4), Rational(1/2), Rational(1,4)]

print(la)
print(ha)

[1/2, 1, -1/2]
[1/4, 1/2, 1/4]


In [14]:
def toZ(coefs): return lambda var : coefs[0]*var**0 + coefs[1]*var**-1 + coefs[2]*var**-2
    
LA_f = toZ(la)
HA_f = toZ(ha)

LA_p = LA_f(z)
LA_n = LA_f(-z)
HA_p = HA_f(z)
HA_n = HA_f(-z)

In [42]:
def denominador(var):
    return LA_p*HA_n - LA_n*HA_p

denominador(z).expand()

z**(-3)

In [39]:
def denominadorA(var):
    return LA_p*HA_n

denominadorA(z).expand()

1/8 - 1/(2*z**2) + 1/(2*z**3) - 1/(8*z**4)

In [41]:
def denominadorB(var):
    return LA_n*HA_p

denominadorB(z).expand()

1/8 - 1/(2*z**2) - 1/(2*z**3) - 1/(8*z**4)

In [45]:
def LS_f(var, m):
    numerador = 2*var**(-m) * HA_n
    denominador = LA_p*HA_n - LA_n*HA_p
    return numerador/denominador

LS_f(z,3).expand()

#LS [1/2, -1, 1/2]

1/2 - 1/z + 1/(2*z**2)

In [44]:
def numerador_LS(var,m):
    return 2*var**(-m) * HA_n

numerador_LS(z,0)

1/2 - 1/z + 1/(2*z**2)

In [46]:
HA_n

1/4 - 1/(2*z) + 1/(4*z**2)

In [24]:
def HS_f(var, m):
    numerador = -2*var**(-m) * LA_n
    denominador = LA_p*HA_n - LA_n*HA_p
    return numerador/denominador

HS_f(z,3).simplify()
#HS = [-1, 2, 1]

-1 + 2/z + z**(-2)

In [47]:
LA_n

1/2 - 1/z - 1/(2*z**2)

## Testando a reconstrução perfeita deste coeficientes


In [28]:
def conv_circular(x, h, verbose=False):
    N = len(x)
    M = len(h)
    w = np.zeros(N)
    for k in range(N):
        if verbose: print(f'w[{k}] = ', end=' ')
        for m in range(M):
            if verbose: print(f'h[{m}] * x[{k-m}] ({h[m]}*{x[k-m]}={h[m] * x[k-m]})', end= '')
            if verbose and m+1 < M: print(' + ', end='')
            w[k] += h[m] * x[k-m]
        if verbose: print(f' = {w[k]}')
    return w

def one_stage_analysis(x, l_analysis, h_analysis): 
    N = len(x)
   
    Xl = conv_circular(x,l_analysis)[::2]
    Xh = conv_circular(x,h_analysis)[::2]
    
    X = np.concatenate([Xl, Xh])
    return X, Xl, Xh

def upsample(x): return np.concatenate([np.insert(x, slice(1, None), 0), np.zeros(1)])

def one_stage_synthesis(X, l_synthesis, h_synthesis, delay=0):
    N = len(X)
    
    Xl_condensed = X[:N//2]
    Xh_condensed = X[N//2:]
    Xl = upsample(Xl_condensed)
    Xh = upsample(Xh_condensed)
    
    vl = conv_circular(Xl, l_synthesis)
    vh = conv_circular(Xh, h_synthesis)
    
    x_delayed = vl + vh
    x = np.roll(x_delayed, -delay)
    
    return x



In [34]:
l_analysis = np.array([1/2, 1, -1/2])
h_analysis = np.array([1/4, 1/2, 1/4])
l_synthesis = np.array([1/2, -1, 1/2])
h_synthesis = np.array([-1, 2, 1])

x = np.array([1,2,3,4,5,6,7,8])

X, Xl, Xh = one_stage_analysis(x, l_analysis, h_analysis)
X



array([5., 3., 5., 7., 6., 2., 4., 6.])

In [35]:
one_stage_synthesis(X, l_synthesis, h_synthesis, delay=3)

array([1., 2., 3., 4., 5., 6., 7., 8.])